In [2]:
!pip install -qU openai-agents

In [3]:
from agents import Agent, Runner, input_guardrail, output_guardrail 
from pydantic import BaseModel

OPENAI_API_KEY is not set, skipping trace export


In [5]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

In [6]:
math_helper = Agent(
    name="Math Helper",
    instructions="You are a math helper. Give hints or explanations for math problems, but don’t solve them completely."
)

In [8]:
@input_guardrail
async def check_if_math(ctx, agent, question):
    # Check if "math" is in the question (simple example)
    if "math" in question.lower():
        return GuardrailFunctionOutput(tripwire_triggered=False)  # Good: It’s about math
    else:
        return GuardrailFunctionOutput(tripwire_triggered=True)   # Bad: Not about math

In [9]:
@output_guardrail
async def no_full_answers(ctx, agent, answer):
    # If the answer has "=", it might be solving the problem
    if "=" in answer:
        return GuardrailFunctionOutput(tripwire_triggered=True)   # Bad: Looks like a full answer
    else:
        return GuardrailFunctionOutput(tripwire_triggered=False)  # Good: No full answer

In [10]:
math_helper.input_guardrails = [check_if_math]
math_helper.output_guardrails = [no_full_answers]

# Step 5: Run the agent with a question
async def run_agent():
    question = "How do I solve 5x = 10?"  # Try asking something!
    try:
        result = await Runner.run(math_helper, question)
        print("Answer:", result.final_output)
    except Exception as e:
        print("Oops! Guardrail stopped it:", str(e))

In [11]:
asyncio.run(run_agent())

Oops! Guardrail stopped it: name 'GuardrailFunctionOutput' is not defined
